In [2]:
from pygame.locals import *
import pygame
import random
import time
import itertools
from PIL import Image, ImageDraw

In [3]:
class Player:
    __id__ = 0
    __step__ = 8
    
    x = 0
    y = 0
    covid = 0
 
    updateCountMax = 2
    updateCount = 0

    def __init__(self, id):
        self.__id__ = id
        self.__step__ = __STEP__ if random.randint(0,1000) > __STAY_HOME__ else 0
        random.seed()
        self.x = random.randint(0, __fieldWidth__)
        self.y = __headerHeight__ + random.randint(0, __fieldHeight__)
        if random.randint(0,1000) < 20:
            self.covid = 1

    def update(self):
        random.seed()
        rnd = random.getrandbits(4)
        rnd_list.append(rnd)
        if rnd & 1 != 0 and (self.x < (__fieldWidth__ - (self.__step__ * 2))):
            self.x = self.x + self.__step__
        if rnd & 2 != 0 and (self.x > self.__step__ * 2):
            self.x = self.x - self.__step__
        if rnd & 4 != 0 and (self.y > __headerHeight__ + self.__step__ * 2):
            self.y = self.y - self.__step__
        if rnd & 8 != 0 and (self.y < (__fieldHeight__ - (self.__step__ * 2))):
            self.y = self.y + self.__step__
        if self.covid > 0:
            self.covid += 1

    def draw(self, surface, image):
        surface.blit(image, (self.x,self.y)) 

In [28]:
class App:
    players = []
    __time__ = 0

    def __init__(self, population = 10):
        self.__time__ = 0
        self._running = True
        self._display_surf = None
        self._image_surf = None
        self._image_result = None
        self._canvas_ = None
        self.windowWidth = __fieldWidth__
        self.windowHeight = __fieldHeight__
        self.players = [Player(id) for id in range(population)] 

    def on_init(self):
        pygame.init()
        self._display_surf = pygame.display.set_mode((self.windowWidth,self.windowHeight), pygame.HWSURFACE)
        
        pygame.display.set_caption('Covid spread simulation')
        self._running = True
        self._image_red = pygame.image.load("red.png").convert()
        self._image_green = pygame.image.load("green.png").convert()
        self._image_yellow = pygame.image.load("yellow.png").convert()
        self._image_result = pygame.Surface((__fieldWidth__, __headerHeight__), pygame.SRCALPHA)
#         self._canvas_ = ImageDraw.Draw(self._image_result)
 
    def on_event(self, event):
        if event.type == QUIT:
            self._running = False

    def on_loop(self):
        for p in self.players:
            p.update()
        pass
    
    def on_render(self):
        self._display_surf.fill((255,255,255))
        for p in self.players:
            if p.covid == 0:
                infects = [p for p in self.players if p.covid > 0 and p.covid < __LATENCE__]
                for i in infects:
                    distance = ((p.x - i.x)**2 + (p.y - i.y)**2)**0.5 
                    if distance < __STEP__ * 3:
                        p.covid = 1
            
            if p.covid > __LATENCE__:
                p.draw(self._display_surf, self._image_yellow)
            elif p.covid > 0:
                p.draw(self._display_surf, self._image_red)
            else:
                p.draw(self._display_surf, self._image_green)
                
        healthy = len([p for p in self.players if p.covid == 0]) / __population__
        infected = len([p for p in self.players if p.covid > 0 and p.covid < __LATENCE__]) / __population__
        immune = len([p for p in self.players if p.covid > __LATENCE__]) / __population__
        
        pygame.draw.line(self._display_surf, (255, 0, 0), (self.__time__, 0), (self.__time__, 100), 1)
#         pygame.draw.line(self._image_result, [255, 0, 0], [(self.__time__, 0), (self.__time__, 100)])
        self._display_surf.blit(self._image_result, (0,0)) 
        
        pygame.display.flip()
        self.__time__ += 1
        
 
    def on_cleanup(self):
        pygame.quit()
 
    def on_execute(self):
        if self.on_init() == False:
            self._running = False
 
        while( self._running ):
            pygame.event.pump()
            keys = pygame.key.get_pressed() 
            if (keys[K_ESCAPE]):
                self._running = False
            self.on_loop()
            self.on_render()

            time.sleep (20.0 / 1000.0);
#         distribution = [(i, l) for v in it.groupBy(rnd_list, v==i for i in range(16))]
        d = []
        for i in range(16):
            d.append((i, len(list([item for item in rnd_list if item == i]))))
        print(d)
        self.on_cleanup()

In [29]:
if __name__ == "__main__" :
    __fieldWidth__ = 1024
    __fieldHeight__ = 768
    __headerHeight__ = 100
    rnd_list = []
    
    __population__ = 100
    __STEP__ = 8
    __LATENCE__ = 320
    __STAY_HOME__ = 750
    
    theApp = App(__population__)
    theApp.on_execute()

[(0, 2810), (1, 2878), (2, 2810), (3, 2899), (4, 2935), (5, 2779), (6, 2886), (7, 2779), (8, 2851), (9, 2802), (10, 2802), (11, 2895), (12, 2778), (13, 2931), (14, 2786), (15, 2879)]
